In [1]:
from maml.apps.bowsr.model.cgcnn import CGCNN
from maml.apps.bowsr.optimizer import BayesianOptimizer
from pymatgen.core.periodic_table import get_el_sp
model = CGCNN()

def expected_radius(struct):
    element_list = struct.composition.chemical_system.split('-')
    element_list = [get_el_sp(e) for e in element_list]
    ele1, ele2 = sorted(element_list, key=lambda x: x.atomic_radius)[:2]
    return ele1.atomic_radius + ele2.atomic_radius

# Load cubic and orthorhombic perovskite

In [2]:
from pymatgen.core import Structure

cubic = Structure.from_file("cubic_perovskite.cif")
orthorhombic = Structure.from_file("orthorhombic_perovskite.cif")

# Distorted structures

In [3]:
compressed_cubic = cubic.copy()
compressed_cubic.scale_lattice(cubic.volume * 0.8)
expanded_cubic = cubic.copy()
expanded_cubic.scale_lattice(cubic.volume * 1.2)

compressed_orthorhombic = orthorhombic.copy()
compressed_orthorhombic.scale_lattice(orthorhombic.volume * 0.8)
expanded_orthorhombic = orthorhombic.copy()
expanded_orthorhombic.scale_lattice(orthorhombic.volume * 1.2)

# Experiment on cubic perovskite

In [4]:
ef_cubic = model.predict_energy(cubic)
print("CGCNN predicted formation energy of cubic perovskite: {} eV/atom".format(ef_cubic))
ef_compressed_cubic = model.predict_energy(compressed_cubic)
print("CGCNN predicted formation energy of compressed cubic perovskite: {} eV/atom".format(ef_compressed_cubic))
ef_expanded_cubic = model.predict_energy(expanded_cubic)
print("CGCNN predicted formation energy of expanded cubic perovskite: {} eV/atom".format(ef_expanded_cubic))

CGCNN predicted formation energy of cubic perovskite: -3.532747268676758 eV/atom
CGCNN predicted formation energy of compressed cubic perovskite: -3.3572254180908203 eV/atom
CGCNN predicted formation energy of expanded cubic perovskite: -3.3126463890075684 eV/atom


In [5]:
compressed_optimizer = BayesianOptimizer(
    model=model, 
    structure=compressed_cubic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
compressed_optimizer.set_bounds()
compressed_optimizer.optimize(n_init=100, n_iter=100, alpha=0.035 ** 2)

In [6]:
radius = max(round(expected_radius(compressed_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_compressed_cubic, _ = compressed_optimizer.get_optimized_structure_and_energy(radius=radius)
print("CGCNN predicted formation energy of relaxed compressed cubic: {} eV/atom".format(model.predict_energy(relaxed_from_compressed_cubic)))

CGCNN predicted formation energy of relaxed compressed cubic: -3.5507938861846924 eV/atom


In [7]:
expanded_optimizer = BayesianOptimizer(
    model=model, 
    structure=expanded_cubic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
expanded_optimizer.set_bounds()
expanded_optimizer.optimize(n_init=100, n_iter=100, alpha=0.035 ** 2)

In [8]:
radius = max(round(expected_radius(expanded_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_expanded_cubic, _ = expanded_optimizer.get_optimized_structure_and_energy(radius=radius)
print("CGCNN predicted formation energy of relaxed expanded cubic: {} eV/atom".format(
    model.predict_energy(relaxed_from_expanded_cubic)))

CGCNN predicted formation energy of relaxed expanded cubic: -3.550915479660034 eV/atom


# Experiment on orthorhombic perovskite

In [9]:
ef_orthorhombic = model.predict_energy(orthorhombic)
print("CGCNN predicted formation energy of orthorhombic perovskite: {} eV/atom".format(ef_orthorhombic))
ef_compressed_orthorhombic = model.predict_energy(compressed_orthorhombic)
print("CGCNN predicted formation energy of compressed orthorhombic perovskite: {} eV/atom".format(ef_compressed_orthorhombic))
ef_expanded_orthorhombic = model.predict_energy(expanded_orthorhombic)
print("CGCNN predicted formation energy of expanded orthorhombic perovskite: {} eV/atom".format(ef_expanded_orthorhombic))

CGCNN predicted formation energy of orthorhombic perovskite: -3.543715476989746 eV/atom
CGCNN predicted formation energy of compressed orthorhombic perovskite: -3.40541410446167 eV/atom
CGCNN predicted formation energy of expanded orthorhombic perovskite: -3.2899346351623535 eV/atom


In [10]:
compressed_optimizer = BayesianOptimizer(
    model=model, 
    structure=compressed_orthorhombic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
compressed_optimizer.set_bounds()
compressed_optimizer.optimize(n_init=100, n_iter=100, alpha=0.035 ** 2)

In [11]:
radius = max(round(expected_radius(compressed_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_compressed_orthorhombic, _ = compressed_optimizer.get_optimized_structure_and_energy(radius=radius)
print("CGCNN predicted formation energy of relaxed compressed orthorhombic: {} eV/atom".format(
    model.predict_energy(relaxed_from_compressed_orthorhombic)))

CGCNN predicted formation energy of relaxed compressed orthorhombic: -3.634078025817871 eV/atom


In [12]:
expanded_optimizer = BayesianOptimizer(
    model=model, 
    structure=expanded_orthorhombic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
expanded_optimizer.set_bounds()
expanded_optimizer.optimize(n_init=100, n_iter=100, alpha=0.035 ** 2)

In [13]:
radius = max(round(expected_radius(expanded_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_expanded_orthorhombic, _ = expanded_optimizer.get_optimized_structure_and_energy(radius=radius)
print("CGCNN predicted formation energy of relaxed expanded orthorhombic: {} eV/atom".format(
    model.predict_energy(relaxed_from_expanded_orthorhombic)))

CGCNN predicted formation energy of relaxed expanded orthorhombic: -3.6496670246124268 eV/atom
